# 4.3.1 EDNETが提供しているEDINET APIから有価証券報告書のデータを取得
※文章はREADMEに記載した論文から抜粋しています

金融庁のEDINET（Electronic Disclosure for Investors’ NETwork）が提供している EDINET API を利用し、本稿の有価証券報告書を取得した。
 
EDINET APIとは、2019年3月に公開された有価価証券報告書データを取得するためのAPI（Application Programming Interface）のことである。
 
これにより、プログラムを用いた機械的なデータ取得をネットワークに過度な負荷をかけることなく実行可能にしている。EDINETAPIを用いたデータ取得までの手順は以下の通りに行った。
 
1.     コロナ前、コロナ過渡期、コロナ後に該当するそれぞれの日付を設定した URL にリクエストし、その日に公開されたJSON式の文書一覧のデータをレスポンスとして受け取る。
2.     レスポンスとして受け取った文書一覧から取得したい有価証券報告書のIDを読み込む。
3.    有価証券報告書のIDを設定したURLをリクエストし、 Zipファイルとして送られてきた有価証券書のデータをレスポンスとして受け取る。

In [52]:
import datetime 
import pandas as pd
import requests as req
from datetime import datetime as dt
import time
import os
import requests as req

def call_edinet_api(data_frame, data_frame_name , start_date, end_date):
    try:
        day_list = make_day_list(start_date, end_date) 
        edicode_values  = make_edicode_values(data_frame)
        industry_securities_report_doc_list_hash = make_industry_securities_report_doc_list_hash(day_list, edicode_values, data_frame)
        edinet_numbers_dir = call_edinet_numbers_dir(data_frame_name)
        write_edinet_numbers_to_txt(industry_securities_report_doc_list_hash,  edinet_numbers_dir)
        industry_securities_report_doc_list_hash = read_txt_to_industry_securities_report_doc_list_hash(edinet_numbers_dir, data_frame)
        sample_data_dir_with_df_name = call_sample_data_dir(data_frame_name)
        download_xbrl_in_zip(industry_securities_report_doc_list_hash,  sample_data_dir_with_df_name)
        print("download finish")
    except Exception as e:
        with open("error.txt", mode="a") as f:
            f.write(f"{datetime.datetime.now()}: {str(e)} \n") 
        raise e

In [53]:
def make_day_list(start_date, end_date):
    
    #取得する書類の日付を指定
    print("start_date：", start_date)
    print("end_day：", end_date)

    period = end_date - start_date
    period = int(period.days)
    day_list = []
    for d in range(period):
        day = start_date + datetime.timedelta(days=d)
        day_list.append(day)

    day_list.append(end_date)
    print("len(day_list):", len(day_list))

    return day_list


def make_edicode_values(data_frame : pd.DataFrame, edinet_col="[EDINETコード]"):
    return list(data_frame[edinet_col].values)


class EdinetDTO:
    
    def __init__(self, edi_code, industry_name, doc_id):
        self.edi_code = edi_code 
        self.industry_name = industry_name
        self.doc_id = doc_id 
    
    def __str__(self):
        return f"edi_code: {self.edi_code},   industry_name: {self.industry_name}, doc_id: {self.doc_id}"


def make_industry_securities_report_doc_list_hash(day_list :list,  edicode_values :list, data_frame: pd.DataFrame,  industry_col="[業種（東証）]", edi_col="[EDINETコード]"):
   
    industry_securities_report_doc_list_hash = {}
    edinet_dto_list  = make_edinet_dto_list(day_list, data_frame, edicode_values, industry_col, edi_col)
    
    #コロナ前, コロナ禍, コロナ後全てあるかカウントを計測
    edi_code_count_hash = {}
    for edinet_dto in edinet_dto_list :
        edi_code  = edinet_dto.edi_code
        if edi_code  not in  edi_code_count_hash :
            edi_code_count_hash[edi_code] = 1
        else :
            edi_code_count_hash[edi_code] += 1
    
    
    #edi_codeをもとに、全ての期間を保有する企業のみを抽出
    industry_securities_report_doc_list_hash = {}
    for  edinet_dto in edinet_dto_list :
        edi_code  = edinet_dto.edi_code
        #コロナ前, コロナ禍, コロナ後全てあるか
        if  edi_code_count_hash[edi_code] == 3:
            
            industry_name  = edinet_dto.industry_name
            doc_id = edinet_dto.doc_id
            if industry_name not in  industry_securities_report_doc_list_hash :
                industry_securities_report_doc_list_hash[industry_name] = [doc_id]
            else:
                industry_securities_report_doc_list_hash[industry_name].append(doc_id)
                
    return  industry_securities_report_doc_list_hash


def make_edinet_dto_list(day_list, data_frame, edicode_values, industry_col, edi_col):
    edinet_dto_list = []
    
    for index, day in enumerate(day_list):
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
        params = {"date": day, "type": 2}
        time.sleep(1)
        res = req.get(url, params=params)
        json_data = res.json()
        
        for num in range(len(json_data["results"])):
            ordinance_code = json_data["results"][num]["ordinanceCode"]
            form_code = json_data["results"][num]["formCode"]

            # 取得したfileが有報かどうか
            if ordinance_code == "010" and form_code == "030000":
                edi_code = json_data["results"][num]["edinetCode"] 
                # 取得したfileが3月期決算企業かどうか
                if  edi_code in edicode_values :
                    industry_name = data_frame[data_frame[edi_col] == edi_code][industry_col].values[0]
                    doc_id = json_data["results"][num]["docID"]
                    edinet_dto = EdinetDTO(edi_code, industry_name, doc_id)
                    print(edinet_dto)
                    edinet_dto_list.append(edinet_dto)
    
    return  edinet_dto_list


def call_edinet_numbers_dir(data_frame_name: str):
    #一時的に一覧取得するためのIDを保存
    
    edinet_numbers_dir = os.getcwd()+  "/EdinetNumbers"
    if not os.path.exists(edinet_numbers_dir) :
        os.mkdir(edinet_numbers_dir)

    edinet_numbers_dir = os.getcwd()+  "/EdinetNumbers"+ f"/{data_frame_name}"
    if not os.path.exists(edinet_numbers_dir) :
        os.mkdir(edinet_numbers_dir)
        
    return edinet_numbers_dir


def write_edinet_numbers_to_txt(industry_securities_report_doc_list_hash,  edinet_numbers_dir):
    for k, v in industry_securities_report_doc_list_hash.items():
        lines = "\n".join(v)
        with open(f'{edinet_numbers_dir}/{k}_edinet_numbers.txt', 'w') as f:
            f.write(lines)
            

def read_txt_to_industry_securities_report_doc_list_hash(edinet_numbers_dir, data_frame, industry_col="[業種（東証）]"):
    
    industry_securities_report_doc_list_hash = {}
    
    industry_list = list(set(data_frame[industry_col].values))
    for industry_name in industry_list:
    
        with open(f'{edinet_numbers_dir}/{industry_name}_edinet_numbers.txt', 'r') as f:
            edinet_numbers = f.read().splitlines()
            industry_securities_report_doc_list_hash[industry_name] = edinet_numbers
    
    return industry_securities_report_doc_list_hash


def call_sample_data_dir(data_frame_name: str):
    
    #Sampleフォルダの作成
    sample_data_dir = os.getcwd()+  "/SampleData"
    if not os.path.exists(sample_data_dir) :
        os.mkdir(sample_data_dir)
        print(f"created: {sample_data_dir}")
    
    sample_data_dir_with_df_name =  sample_data_dir + f"/{data_frame_name}"
    if not os.path.exists(sample_data_dir_with_df_name) :
        os.mkdir(sample_data_dir_with_df_name)
    return sample_data_dir_with_df_name


def download_xbrl_in_zip(industry_securities_report_doc_list_hash,  sample_data_dir_with_df_name):
    
    sum_files_len = sum(map(len, industry_securities_report_doc_list_hash.values()))
    v_len = 0
    
    for k, v in industry_securities_report_doc_list_hash.items():
        
        filepath_edinet = call_industry_dir(sample_data_dir_with_df_name, k)
        v_len += len(v)
        for index, doc_id in enumerate(v):
            print(f"{doc_id} :{ index + 1} /{len(v)}  | {v_len} /{sum_files_len}")
            url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + doc_id
            params = {"type": 1}
            filename = filepath_edinet + "/"  + doc_id + ".zip"
            if os.path.exists(filename) :
                continue
            else:
                #負荷を与えないために2秒間待つ
                time.sleep(2)
                res = req.get(url, params=params, stream=True)

                if res.status_code == 200:
                    with open(filename, 'wb') as file:
                        for chunk in res.iter_content(chunk_size=1024):
                            file.write(chunk)
                            

def call_industry_dir( sample_data_dir_with_df_name, industry_name):
    industry_dir =   sample_data_dir_with_df_name+  f"/{industry_name}"
    if not os.path.exists(industry_dir) :
        os.mkdir(industry_dir)
        print(f"created: {industry_dir}")
    return industry_dir

In [54]:
#今回は新型コロナウイルス感染症を意識した期間を設定
  # コロナ前(2019年3月期決算: 2018年4月1日~2019年3月31日)
  # コロナ過渡期(2019年4月~2020年3月)
  # コロナ後(2020年4月~2021年3月)
start_date = datetime.date(2018, 4, 1)
end_date = datetime.date(2021, 3,31)
day_list = make_day_list(start_date, end_date)

start_date： 2018-04-01
end_day： 2021-03-31
len(day_list): 1096


In [55]:
#今回は2021年度時点での3月期決算企業のEDINETコードが記載されたcsvファイルを使用(hirenketsu or renketsuを指定)
data_frame_name = "hirenketsu"
data_frame = pd.read_csv(f"./EdinetIdxFiles/edinet_{data_frame_name}.csv", skiprows=4)

In [56]:
call_edinet_api(data_frame, data_frame_name, start_date, end_date)

start_date： 2018-04-01
end_day： 2021-03-31
len(day_list): 1096
edi_code: E03266,   industry_name: 小売業, doc_id: S100D5X5
edi_code: E02966,   industry_name: 卸売業, doc_id: S100D6H4
edi_code: E01060,   industry_name: 化　学, doc_id: S100D5XW
edi_code: E03160,   industry_name: 小売業, doc_id: S100D6MH
edi_code: E26327,   industry_name: 保険業, doc_id: S100D6IR
edi_code: E04217,   industry_name: 陸運業, doc_id: S100D6TB
edi_code: E03807,   industry_name: 証券、商品先物取引業, doc_id: S100D6G1
edi_code: E09753,   industry_name: その他金融業, doc_id: S100D76Y
edi_code: E02750,   industry_name: 卸売業, doc_id: S100D6GQ
edi_code: E04990,   industry_name: サービス業, doc_id: S100D83N
edi_code: E02753,   industry_name: 卸売業, doc_id: S100D81M
edi_code: E05112,   industry_name: 小売業, doc_id: S100D8JR
edi_code: E00737,   industry_name: パルプ・紙, doc_id: S100D8P1
edi_code: E31012,   industry_name: 食料品, doc_id: S100D8Q9
edi_code: E33435,   industry_name: 不動産業, doc_id: S100D8UL
edi_code: E01033,   industry_name: 化　学, doc_id: S100D8FE
edi_code: 

KeyboardInterrupt: 